# Week3_4 Assignment

## [BASIC](#Basic) 
- Encoder & Decoder Layer 코드를 직접 필사하고 각 함수에 주석을 달 수 있다. 

## [CHALLENGE](#Challenge)
- 텐서의 크기(shape)를 계산할 수 있다. 

## [ADVANCED](#Advanced)
- 완성된 transformer 모델의 모든 학습 가능한 파라미터 이름과 크기(shape)를 출력할 수 있다.

### Informs
이번 과제에서는 "[Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)"의 코드를 필사해본다.   
"Annotated Transformer"는 "Attention is all you need" 논문에서 제안한 transformer 모델을 pytorch 라이브러리로 직접 구현한다.   
코드 필사를 통해 다음을 배울 수 있다.    
- Encoder, Decoder 구조
- Attention Mechanism
- "residual connection", "layer normalization" 등의 구조 

코드 필사를 시작하기 앞서, transformer 모델의 최종 구조를 살펴보자.    

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_full.png?raw=true" width="500" align="center"/>

최종 모델은 `EncoderDecoder()` 클래스에 여러 인스턴스를 생성자의 입력 파라미터로 넣어 생성한다.    
앞으로 우리는 `EncoderDecoder()` 클래스와 같은 여러 클래스들을 구현하고 연결할 것이다. 따라서 대략적인 클래스간의 관계를 살펴보고 이해한다면 보다 큰 그림을 가지고 코드 필사를 할 수 있을 것이다. 

Transformer 모델은 크게 4가지 클래스로 구현된다.    
- Frame
    - frame 역할을 하는 `EncoderDecoder` 클래스
- Input Embedding & Encoding
    - 입력값을 벡터화하는 `Embeddings`, `PositionalEncoding`
- Encoder & Decoder
    - 각 6개 layer를 갖고 있는 `Encoder`, `Decoder`
    - layer 1층을 구현한 `EncoderLayer`, `DecoderLayer`
- Sublayer
    - `EncoderLayer`, `DecoderLayer` 내부에서 사용되는 Sublayer 클래스인 `MultiHeadAttiontion`, `PositionwiseFeedForward`
    - Sublayer 클래스들을 연결하는 `SublayerConnection`
    
아래 좌측 도식에서 각 클래스의 색상은 아래 우측 도식(transformer 구조)의 색상과 맵핑되어 있다.    
각 클래스의 역할과 클래스 간 연결 관계를 생각하면서 transformer를 코드로 구현해보자.   


<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_map.png?raw=true" width="400" height="400" align="left"/>
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_transformer.png?raw=true" width="300" height="400" align="right"/>



In [95]:
import os
import sys
import pandas as pd
import numpy as np 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import math, copy, time
import random

## Basic

### Frame
- `EncoderDecoder`

아래 도식은 `EncoderDecoder` 클래스의 `forward()`, `encode()`, `decode()` 메소드를 도식화 한 것이다.    
 
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_encoderdecoder.png?raw=true" width=500>


- `Generator`

In [96]:
class EncoderDecoder(nn.Module): 
    """
    표준 인코더 아키텍쳐, 많은 모델의 기본
    """
    
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed #src = 소스
        self.tgt_embed = tgt_embed #tgt = 타겟
        self.generator = generator 
 
    
    
    def forward(self, src, tgt, src_mask, tgt_mask):
         return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask) # intput (src,src_mask)=>src_embed,src_mask => enc => enc output(memory), src_mask, intput(tgt, tgt_mask) => tgt_embed=> decode
      
    
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src),src_mask) #src =>src_embed,src_mask => enc
    
    
    def decode(self, memory, src_mask, tgt, tgt_mask): #enc output(memory) ,tgt => tgt_embed => decode
        return self.decoder(self.tgt_embed(tgt),memory, src_mask, tgt_mask)

In [97]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step. 레이어 마지막에 linear와 softmax 부분"
    
    def __init__(self, d_model, vocab):
        super(Generator,self).__init__()
        self.proj = nn.Linear(d_model,vocab) 

    
    
    def forward(self, x):
        return F.log_sfotmax(self.proj(x), dim=-1) #모델에서 사용하는 부분을 찾지 못했다 어디서 사용되는건지..


### Encoder
- `Encoder`
- `EncoderLayer`
- `SublayerConnection`
- Reference
    - Layer Normalization
        - [한국어 설명](https://yonghyuc.wordpress.com/2020/03/04/batch-norm-vs-layer-norm/)
        - [torch official docs](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html)
    - Residual Connection
        - [한국어 설명](https://itrepo.tistory.com/36)
    - pytorch ModuleList
        - [torch official docs](https://pytorch.org/docs/1.9.1/generated/torch.nn.ModuleList.html)


In [98]:
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)]) #n개 만큼동일한 레이어를 생성


In [99]:
class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N) #n개의 layer생성
        self.norm = LayerNorm(layer.size) #layer normalization , 배치단위가 아닌 인풋된 피처 레이어 단위로 정규화
    
    
    
    def forward(self, x, mask):  

        for layer in self.layers: #입력값과 마스크를 레이어들을 통과시킨다
            x = layer(x, mask)
        return self.norm(x)


In [100]:
class LayerNorm(nn.Module): #layer normalization
    
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features)) #nn.parameter => 레이어가 아닌 파라미터값만 가지고있음  torch.ones => 1로 이루어진 텐서  
        self.b_2 = nn.Parameter(torch.zeros(features)) # torch.zeros => 0으로 이루어진 텐서
        self.eps = eps
   
    
    
    def forward(self, x):
        mean = x.mean(-1, keepdim=True) #평균, 구함 입력된 피쳐단위로 , 차원유지 유지
        std = x.std(-1, keepdim=True) #표준편차, 위와같음
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2 # ones와 zeros를 왜 쓰는지 모르겠음
   

In [101]:
class SublayerConnection(nn.Module): #정규화를 해주고 출력값에 입력값을 더해준다, x + 와 sublayer를 하기위해 두값의 차원을 맞춰줘야한다 (이게어느부분이지?)
    
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size) #norm 생성
        self.dropout = nn.Dropout(dropout) # 드랍아웃은 왜들어가지?
   
    
    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x))) #입력값 x와 sublayer를 연결
  

In [102]:
class EncoderLayer(nn.Module):
    
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2) #서브레이어를 두개만듬
        self.size = size

        
    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask)) #쿼리 키 밸류, 마스크 #forward는 선언안하고 써도되는건가??? =>forward() 함수는 model 객체를 데이터와 함께 호출하면 자동으로 실행이됩니다.
        return self.sublayer[1](x, self.feed_forward)#인풋 x -> 셀프어텐션 -> x+ ountput x -> 피드포워드 -> x + output x

### Decoder
- `Decoder`
- `DecoderLayer`

In [80]:
class Decoder(nn.Module): #인코더와 동일 레이어숫자, 레이어 정규화
    
    def __init__(self, layer, N):
        super(Decoder, self).__init__() 
        self.layers = clones(layer, N) 
        self.norm = LayerNorm(layer.size) 
 
    
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:#입력값과 마스크를 레이어들을 통과시킨다
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [81]:
class DecoderLayer(nn.Module):

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
     
    
    
    def forward(self, x, memory, src_mask, tgt_mask): #입력값을 출력값에 더해주고 2번째에선 인코더의 출력값과 마스크도 넣어준다 
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask)) #쿼리 키 밸류, 마스크
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask)) #쿼리(이전 디코더 값의 출력) memory 키 memory 밸류, src의 마스크
        return self.sublayer[2](x, self.feed_forward) # 
  

### Sublayer
- `attention` 함수

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_attention.png?raw=true" width="500" align="center"/>  

- `MultiHeadedAttention`
- `PositionwiseFeedForward`

### Challenge


### Q1. 위 도식에 따라 `score`, `p_attn`, `attention` 을 구하라 

In [110]:
def attention(query, key, value, mask=None, dropout=None): #쿼리 키 밸류
#Query :  디코더의 이전 레이어 hidden state , 영향을 받는 디코더의 토큰, K : 인코더의 output state , 영향을 주는 인코더의 토큰들 ,V : 인코더의 output state , 그 영향에 대한 가중치가 곱해질 인코더 토큰들
#self attention은 q=k=v
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)        
    return torch.matmul(p_attn, value), p_attn #attention , p_attn
  

###Q2. query, key, value가 모두 (m, d_k) shape의 matrix라고 가정할 때, `score`, `p_attn`, `attention`의 shape을 각각 구하라
- score : m,m
- p_attn : m,m 
- attention : m,d_k

### (아래의 **Q3을 먼저 풀고 돌아오세요**) Q4.  query, key, value가 모두 (12, 8, 1, 64) shape의 tensor라고 가정할 때 , `score`, `p_attn`, `attention`의 shape을 각각 구하라

- score : 12, 8, 1, 1
- p_attn : 12, 8, 1, 1
- attention : 12, 8, 1, 64

- `MultiHeadedAttention`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_multihead.png?raw=true" width="300" align="center"/>  

In [83]:
class MultiHeadedAttention(nn.Module):
    """
    한번의 어텐션을 하는 것 보다 어텐션을 병렬로 여러번 사용하는것이 더 효과적이다
    d_model의 차원을 num_heads개로 나누어 그 값의 차원을 갖는 q,k,v에 대해 num_heads개의 병렬 어텐션을 수행한다 (어텐션 헤드들의 가중치 행렬값은 전부 다름)
    다른 시각으로 어텐션 정보를 수집하게된다 
    """
    def __init__(self, h, d_model, dropout=0.1): #h: 헤드숫자, d_model 트랜스포머의 모든 층의 출력 차원 실제 논문에선 512
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0        
    
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
     
    
    
    def forward(self, query, key, value, mask=None): #멀티어텐션으로 나눠서 계산하고 다시 합친다는건 알겠는데 모르는 코드가 너무많다
        if mask is not None:
            # 모든 헤드에 동일한 마스크 적용
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
                
        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) 어텐션을 적용 , Apply attention on all the projected vectors in batch. ??
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. ????          
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)# \ = 다음줄이랑 이어지게 해줌 
        jj=nn.Linear(512, 512)

        return self.linears[-1](x)
  
            

### Q3.  query, key, value가 모두 (12, 512) shape의 matrix이고, h 값이 8 이라고 가정할 때, 아래 값의 shape을 각각 구하라

- `d_k` (d_k = d_model // h) : 64
- `nn.Linear(d_model, d_model)(query)` : 12, 512
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k)` : 12, 1, 8, 64
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k).transpose(1,2)` : 12, 8, 1, 64

- `PositionwiseFeedForward`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pwff.png?raw=true" width="300" align="center"/>  

In [84]:
class PositionwiseFeedForward(nn.Module): #개별 단어마다 적용되기 때문에 position-wise이다 (코드가 왜 이렇게 하는건지 모르겠다) 논문에선 d_model 512 , d_ff 2048
    def __init__(self, d_model, d_ff, dropout=0.1): 
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x)))) 

### Input Embedding & Encoding
- `Embeddings`
    - [pytorch official docs](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [85]:
class Embeddings(nn.Module):
  
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model
    
    
    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)
    

- `PositionalEncoding`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pe.png?raw=true" width="500" align="center"/>  

- `position` 변수 설명
    - 모든 position (=최대 토큰 개수)의 값을 갖고 있는 matrix
- `div_term` 변수 설명

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_div.png?raw=true" width="500" align="center"/>  
- `Embedding` + `Encoding` 도식화 

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_emb_enc.png?raw=true" width="400" align="center"/>  


In [87]:
class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model, dropout, max_len = 5000): #포지션값을 계산하고 입력값과 더해서 인코더나 디코더의 입력으로 넘겨주게된다
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model) #계산방식은다시한번 살펴봐야할것같다
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
      
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)
      

### Q4.  max_len이 512이고, d_model이 512라고 가정할 때, `position`과 `div_term`의 shape을 구하라

- `position` : 512,1
- `div_term` : 256
- `position * div_term` : 512,256

### Advanced

### Finally Build Model
- Xavier Initialization
    - [한국어 자료](https://huangdi.tistory.com/8)
    - [pytorch official docs](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.xavier_uniform_)

In [88]:
def make_model(src_vocab, tgt_vocab,  #왜 모델생성에서는 class를 안쓸까?
               N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters." #연결된다는걸 알겠지만 하나하나 순서를 따라가면서 다시봐야할것같다
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

            

In [89]:
model = make_model(10,10)

1111 64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


### Q5. 위 코드로 만든 모델의 모든 파라미터의 이름과 크기 (shape) 을 출력하라

In [94]:
for name, param in model.named_parameters():
    print(param.size())
    print(name)

torch.Size([512, 512])
encoder.layers.0.self_attn.linears.0.weight
torch.Size([512])
encoder.layers.0.self_attn.linears.0.bias
torch.Size([512, 512])
encoder.layers.0.self_attn.linears.1.weight
torch.Size([512])
encoder.layers.0.self_attn.linears.1.bias
torch.Size([512, 512])
encoder.layers.0.self_attn.linears.2.weight
torch.Size([512])
encoder.layers.0.self_attn.linears.2.bias
torch.Size([512, 512])
encoder.layers.0.self_attn.linears.3.weight
torch.Size([512])
encoder.layers.0.self_attn.linears.3.bias
torch.Size([2048, 512])
encoder.layers.0.feed_forward.w_1.weight
torch.Size([2048])
encoder.layers.0.feed_forward.w_1.bias
torch.Size([512, 2048])
encoder.layers.0.feed_forward.w_2.weight
torch.Size([512])
encoder.layers.0.feed_forward.w_2.bias
torch.Size([512])
encoder.layers.0.sublayer.0.norm.a_2
torch.Size([512])
encoder.layers.0.sublayer.0.norm.b_2
torch.Size([512])
encoder.layers.0.sublayer.1.norm.a_2
torch.Size([512])
encoder.layers.0.sublayer.1.norm.b_2
torch.Size([512, 512])
enco